In [3]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('Analyse').getOrCreate()

25/03/14 16:32:17 WARN Utils: Your hostname, affes-GF63-Thin-11SC resolves to a loopback address: 127.0.1.1; using 192.168.0.9 instead (on interface wlo1)
25/03/14 16:32:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/14 16:32:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

In [6]:
df_pyspark=spark.read.option('header','true').csv('big4_financial_risk_compliance.csv')

In [7]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [8]:
df_pyspark

DataFrame[Year: string, Firm_Name: string, Total_Audit_Engagements: string, High_Risk_Cases: string, Compliance_Violations: string, Fraud_Cases_Detected: string, Industry_Affected: string, Total_Revenue_Impact: string, AI_Used_for_Auditing: string, Employee_Workload: string, Audit_Effectiveness_Score: string, Client_Satisfaction_Score: string]

In [9]:
df_pyspark.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Firm_Name: string (nullable = true)
 |-- Total_Audit_Engagements: string (nullable = true)
 |-- High_Risk_Cases: string (nullable = true)
 |-- Compliance_Violations: string (nullable = true)
 |-- Fraud_Cases_Detected: string (nullable = true)
 |-- Industry_Affected: string (nullable = true)
 |-- Total_Revenue_Impact: string (nullable = true)
 |-- AI_Used_for_Auditing: string (nullable = true)
 |-- Employee_Workload: string (nullable = true)
 |-- Audit_Effectiveness_Score: string (nullable = true)
 |-- Client_Satisfaction_Score: string (nullable = true)



In [10]:
df_pyspark.show(3,truncate=False)

+----+---------+-----------------------+---------------+---------------------+--------------------+-----------------+--------------------+--------------------+-----------------+-------------------------+-------------------------+
|Year|Firm_Name|Total_Audit_Engagements|High_Risk_Cases|Compliance_Violations|Fraud_Cases_Detected|Industry_Affected|Total_Revenue_Impact|AI_Used_for_Auditing|Employee_Workload|Audit_Effectiveness_Score|Client_Satisfaction_Score|
+----+---------+-----------------------+---------------+---------------------+--------------------+-----------------+--------------------+--------------------+-----------------+-------------------------+-------------------------+
|2020|PwC      |2829                   |51             |123                  |39                  |Healthcare       |114.24              |No                  |57               |5.8                      |8.4                      |
|2022|Deloitte |3589                   |185            |30                   |60

In [11]:
df_pyspark.columns

['Year',
 'Firm_Name',
 'Total_Audit_Engagements',
 'High_Risk_Cases',
 'Compliance_Violations',
 'Fraud_Cases_Detected',
 'Industry_Affected',
 'Total_Revenue_Impact',
 'AI_Used_for_Auditing',
 'Employee_Workload',
 'Audit_Effectiveness_Score',
 'Client_Satisfaction_Score']

In [12]:
df_pyspark.dtypes

[('Year', 'string'),
 ('Firm_Name', 'string'),
 ('Total_Audit_Engagements', 'string'),
 ('High_Risk_Cases', 'string'),
 ('Compliance_Violations', 'string'),
 ('Fraud_Cases_Detected', 'string'),
 ('Industry_Affected', 'string'),
 ('Total_Revenue_Impact', 'string'),
 ('AI_Used_for_Auditing', 'string'),
 ('Employee_Workload', 'string'),
 ('Audit_Effectiveness_Score', 'string'),
 ('Client_Satisfaction_Score', 'string')]

In [13]:
df_pyspark=df_pyspark.drop(*['Industry_Affected','Total_Revenue_Impact','AI_Used_for_Auditing','Employee_Workload','Audit_Effectiveness_Score','Client_Satisfaction_Score'])

In [14]:
df_pyspark.show()

+----+-------------+-----------------------+---------------+---------------------+--------------------+
|Year|    Firm_Name|Total_Audit_Engagements|High_Risk_Cases|Compliance_Violations|Fraud_Cases_Detected|
+----+-------------+-----------------------+---------------+---------------------+--------------------+
|2020|          PwC|                   2829|             51|                  123|                  39|
|2022|     Deloitte|                   3589|            185|                   30|                  60|
|2020|          PwC|                   2438|            212|                  124|                  97|
|2021|          PwC|                   2646|            397|                   55|                  97|
|2020|          PwC|                   2680|            216|                   99|                  46|
|2023|     Deloitte|                    818|            448|                   10|                  30|
|2022|     Deloitte|                   1199|            148|    

In [15]:
df_pyspark=df_pyspark.na.drop(how='any', subset=['Firm_Name']).show() #Delete any ligne where firmName is null

+----+-------------+-----------------------+---------------+---------------------+--------------------+
|Year|    Firm_Name|Total_Audit_Engagements|High_Risk_Cases|Compliance_Violations|Fraud_Cases_Detected|
+----+-------------+-----------------------+---------------+---------------------+--------------------+
|2020|          PwC|                   2829|             51|                  123|                  39|
|2022|     Deloitte|                   3589|            185|                   30|                  60|
|2020|          PwC|                   2438|            212|                  124|                  97|
|2021|          PwC|                   2646|            397|                   55|                  97|
|2020|          PwC|                   2680|            216|                   99|                  46|
|2023|     Deloitte|                    818|            448|                   10|                  30|
|2022|     Deloitte|                   1199|            148|    